In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

# 혐오 발언 탐지 모델 로드
MODEL_NAME = "smilegate-ai/kor_unsmile"
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)
MODEL = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
MODEL.eval()

# GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL.to(device)

# 혐오 발언 분석 함수 정의 (토큰 길이 제한 추가)
def predict_hate_speech(text):
    inputs = TOKENIZER(text, return_tensors="pt", truncation=True, padding=True, max_length=300)  # 최대 300 토큰
    inputs = {key: value.to(device) for key, value in inputs.items()}  # GPU로 이동
    with torch.no_grad():
        outputs = MODEL(**inputs)
    scores = torch.sigmoid(outputs.logits)  # 확률값 변환 (각 클래스의 가능성)
    hate_score = scores[0][0].item()  # 혐오 발언 점수 (0~1 사이 값)
    return hate_score  # 높은 값일수록 혐오적

# CSV 데이터 로드
file_path = "daily_conversation_generated_10000.csv"
df = pd.read_csv(file_path)

# 혐오 발언 점수 계산 및 필터링 (임계값 0.5 이상인 문장 제거)
threshold = 0.01
df["hate_score"] = df["generated_dialogue"].apply(lambda x: predict_hate_speech(str(x)) if isinstance(x, str) else 0)  # 문자열이 아닌 경우 예외 처리
filtered_df = df[df["hate_score"] < threshold]  # 혐오 발언 포함 문장 제외

# 결과 확인
filtered_df.to_csv("filtered_hate_speech_data.csv", index=False)
print("Filtered data saved as 'filtered_hate_speech_data.csv'")


Filtered data saved as 'filtered_hate_speech_data.csv'
